In [1]:
from dataloader import SeqDataLoader
import numpy as np

num_folds = 20
data_dir = 'data/data2013/eeg_fpz_cz'
classes = ['W', 'N1', 'N2', 'N3', 'REM']
checkpoint_dir = 'checkpoints-seq2seq-sleep-EDF'
output_dir = 'outputs_2013/outputs_eeg_fpz_cz'
NUM_CLASS= 7
fold_idx = 0

kargs = {
    'epochs': 120,
    'batch_size': 20,
    'num_units': 128,
    'embed_size': 10,
    'input_depth': 3000,
    'n_channels': 100,
    'bidirectional': False,
    'use_attention': True,
    'lstm_layers': 2,
    'attention_size': 64,
    'beam_width': 4,
    'use_beamsearch_decode': False,
    'max_time_step': 10,
    'output_max_length': 10 + 2,
    'akara2017': True,
    'test_step': 5
}

data_loader = SeqDataLoader(data_dir, num_folds, fold_idx, classes=classes)
X_train, y_train, X_test, y_test = data_loader.load_data(seq_len=kargs['max_time_step'])
print(X_train.shape)
X_train = np.reshape(X_train,[X_train.shape[0]*X_train.shape[1],-1])
print(X_train.shape)


========== [Fold-0] ==========

Load training set:
Loading data/data2013/eeg_fpz_cz\SC4022E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4042E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4162E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4092E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4561F0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4481F0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4012E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4672G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4292G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4212E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4331F0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4821G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4671G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4702E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4822G0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4761E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4021E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4312E0.npz ...
Loading data/data2013/eeg_fpz_cz\SC4032E0.npz ..

In [3]:
import torch
X_train=torch.tensor(X_train[:X_train.shape[0]//4])
y_train=torch.tensor(y_train[:y_train.shape[0]//4])
print('feature')
print(X_train.shape)
print(X_train[:10])
print('label')
print(y_train.shape)
print(y_train[:10])

c:\Users\user\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


feature
torch.Size([46122, 3000])
tensor([[-1.1789e-01, -5.1045e-01, -1.5799e-03,  ..., -5.9769e-01,
         -5.1045e-01, -2.4875e-01],
        [-1.4410e+00,  2.0158e-01,  1.0608e-01,  ..., -6.3881e-01,
         -9.4441e-01, -6.1971e-01],
        [-4.3664e-01, -6.5547e-01, -4.7120e-01,  ..., -1.0701e+00,
         -1.0355e+00, -1.1622e+00],
        ...,
        [-2.1498e-01,  6.3238e-01, -2.7854e-01,  ..., -1.7932e+00,
         -1.1683e+00, -1.1789e+00],
        [-8.5821e-01, -1.1031e+00, -9.0718e-01,  ..., -1.1031e+00,
         -4.9912e-01, -7.0315e-01],
        [-2.2816e+00, -1.2748e+00, -3.1054e+00,  ...,  3.9107e-01,
          9.4026e-01,  7.2058e-01]])
label
torch.Size([4612, 10])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [4, 4, 1, 1, 0, 1, 1, 2, 2, 2],
        [0, 0, 0, 0, 1, 2, 1, 1, 2, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
        [2, 3, 3, 3, 2, 3, 3, 2, 2, 2],
        [4, 4, 4, 4, 4, 4

In [5]:
import os
path, channel_ename = os.path.split(data_dir)

char2numY = dict(list(zip(classes, list(range(len(classes))))))        
traindata_dir = os.path.join(os.path.abspath(os.path.join(data_dir, os.pardir)),'traindata/')

char2numY['<SOD>'] = len(char2numY)
char2numY['<EOD>'] = len(char2numY)
num2charY = dict(list(zip(list(char2numY.values()), list(char2numY.keys()))))
fname = os.path.join(traindata_dir,'trainData_'+channel_ename+'_SMOTE_all_10s_f'+str(fold_idx)+'.npz')

nums = []
for cl in classes:
    nums.append(len(np.where(y_train == char2numY[cl])[0]))

In [8]:
from imblearn.over_sampling import SMOTE

n_osamples = nums[2] - 7000
ratio = {0: n_osamples if nums[0] < n_osamples else nums[0], 1: n_osamples if nums[1] < n_osamples else nums[1],
            2: nums[2], 3: n_osamples if nums[3] < n_osamples else nums[3], 4: n_osamples if nums[4] < n_osamples else nums[4]}

sm = SMOTE(random_state=12, sampling_strategy=ratio)
X_train, y_train = sm.fit_resample(X_train, y_train)
print(n_osamples)
print(ratio)
print(X_train.shape)
print(y_train.shape)

57766
{0: 61827, 1: 57766, 2: 64766, 3: 57766, 4: 57766}
(299891, 3000)
(299891,)


In [9]:
X_train = X_train[:(X_train.shape[0] // kargs['max_time_step']) * kargs['max_time_step'], :]
y_train = y_train[:(X_train.shape[0] // kargs['max_time_step']) * kargs['max_time_step']]

X_train = np.reshape(X_train,[-1,X_test.shape[1],X_test.shape[2]])
y_train = np.reshape(y_train,[-1,y_test.shape[1],])

# shuffle training data_2013
permute = np.random.permutation(len(y_train))
X_train = np.asarray(X_train)
X_train = X_train[permute]
y_train = y_train[permute]

print(X_train.shape)
print(y_train.shape)

(29989, 10, 3000)
(29989, 10)


In [10]:
# add '<SOD>' to the beginning of each label sequence, and '<EOD>' to the end of each label sequence (both for training and test sets)
y_train= [[char2numY['<SOD>']] + [y_ for y_ in date] + [char2numY['<EOD>']] for date in y_train]
y_train = np.array(y_train)


y_test= [[char2numY['<SOD>']] + [y_ for y_ in date] + [char2numY['<EOD>']] for date in y_test]
y_test = np.array(y_test)

print(y_train.shape)

print(('The training set after oversampling: ', classes))
for cl in classes:
    print((cl, len(np.where(y_train==char2numY[cl])[0])))

(29989, 12)
('The training set after oversampling: ', ['W', 'N1', 'N2', 'N3', 'REM'])
('W', 61827)
('N1', 57766)
('N2', 64766)
('N3', 57766)
('REM', 57765)


In [11]:
char2numY

{'W': 0, 'N1': 1, 'N2': 2, 'N3': 3, 'REM': 4, '<SOD>': 5, '<EOD>': 6}

In [44]:
import torch.nn as nn
import torch

# Define the embedding layer
embedding_layer = nn.Embedding(num_embeddings=len(char2numY), embedding_dim=10)

# Use the embedding layer
decoder_emb_inputs = embedding_layer(torch.tensor(y_train[:, :-1]))

# Get the embedding matrix (decoder_embedding)
decoder_embedding = embedding_layer.weight.data

In [45]:
print(y_train[:, :-1].shape)
print(decoder_emb_inputs.shape)
print(decoder_embedding.shape)

(29989, 11)
torch.Size([29989, 11, 10])
torch.Size([7, 10])


attention function f(⋅)
attention probability α_i : 각각의 hidden state의 중요도에 관한 것
scores와 hidden states를 곱해서 weighted combination c_t 를 구함

f(h_(t-1),e_i )=tanh⁡(W_h h_(t-1)+W_e e_i )
α_i=softmax(f(h_(t-1),e_i ))
c_t=∑_(i=0)^n▒〖α_i e_i 〗
α_i:hidden state의 part i에 대한 중요도

즉, 매 time step t 마다, attiontion layer는 f(⋅)을 계산함:* encoder 의 hidden state e_i 와 decoder의 hidden state h_(t-1)의 조합에 tanh 씌운거
	그러고 나면, 이걸 softmax 에 넣어서, n개의 parts에 대해서 α_i를 계산함
	마지막으로, 모든 벡터 e_i 와 그에 해당하는 α_i의 attention module c_t를 계산함


In [1]:
decoder_emb_inputs.view(decoder_emb_inputs.shape(0), -1).shape

NameError: name 'decoder_emb_inputs' is not defined

In [43]:
a = torch.ones((29989, 256))
d = torch.ones((29989,1))+4
decoder_embedding.view(29989, -1)
torch.cat([a, d], dim=1).shape

torch.Size([29989, 257])

In [27]:
print(y_train.shape)
print(y_train)
# decoder input
print(y_train[:, :-1].shape)
print(y_train[:, :-1])
# label
print(y_train[:, 1:].shape)
print(y_train[:, 1:])

# input_dim=len(char2numY), output_dim=hparams['embed_size']
print(len(char2numY))
print(kargs['embed_size'])

(29989, 12)
[[5 2 2 ... 2 2 6]
 [5 3 3 ... 3 3 6]
 [5 2 2 ... 2 2 6]
 ...
 [5 0 0 ... 0 0 6]
 [5 4 4 ... 4 4 6]
 [5 0 0 ... 0 0 6]]
(29989, 11)
[[5 2 2 ... 2 2 2]
 [5 3 3 ... 3 3 3]
 [5 2 2 ... 2 2 2]
 ...
 [5 0 0 ... 0 0 0]
 [5 4 4 ... 4 4 4]
 [5 0 0 ... 0 0 0]]
(29989, 11)
[[2 2 2 ... 2 2 6]
 [3 3 3 ... 3 3 6]
 [2 2 2 ... 2 2 6]
 ...
 [0 0 0 ... 0 0 6]
 [4 4 4 ... 4 4 6]
 [0 0 0 ... 0 0 6]]
7
10


In [18]:
NUM_CLASSES = 5
import tensorflow as tf
decoder_input = tf.one_hot(char2numY.get('<SOD>'), depth=NUM_CLASSES+2)
decoder_input = tf.multiply(
tf.ones((X_train.shape[0],NUM_CLASSES+2),dtype=tf.float32),
decoder_input) # <SOD> 로 시작
print(decoder_input.shape)
print(decoder_input[:10])

(29989, 7)
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]], shape=(10, 7), dtype=float32)
